<a href="https://colab.research.google.com/github/bishair/Pirna/blob/main/G10-AddedDates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from math import sqrt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
import matplotlib.pyplot as plt
from IPython.display import display
import joblib


In [3]:
uploaded_file = files.upload()

Saving riverL.xlsx to riverL (2).xlsx


In [4]:
# Read the groundwater level data
gw_data = pd.read_excel('groundwater.xlsx')
gw_data.dropna(inplace=True)
gw_data['Date'] = pd.to_datetime(gw_data['Date'], format='%d/%m/%Y %H:%M')
print(gw_data)

# Read the river water level data
river_data = pd.read_excel('riverL.xlsx')
river_data['Date'] = pd.to_datetime(river_data['Date'], format='%d/%m/%Y %H:%M')

# Merge the two datasets on the 'Date' column
merged_data = pd.merge_asof(gw_data.sort_values('Date'), river_data.sort_values('Date'), on='Date', direction='nearest')

# Set 'Date' as the index
merged_data.set_index('Date', inplace=True)

resampled_data = merged_data.resample('H').ffill()
resampled_data = resampled_data[1:]
print(resampled_data)

'''
has_nan = resampled_data.isna().any().any()
print(has_nan)
total_nan_count = resampled_data.isna().sum().sum()
print(total_nan_count)
nan_per_column = resampled_data.isna().any()
print(nan_per_column)
nan_count_per_column = resampled_data.isna().sum()
print(nan_count_per_column)'''


                     Date      G10
0     2015-01-30 12:18:00  110.722
1     2015-01-30 13:18:00  110.720
2     2015-01-30 14:18:00  110.718
3     2015-01-30 15:18:00  110.715
4     2015-01-30 16:18:00  110.714
...                   ...      ...
17533 2017-01-30 10:19:00  109.456
17534 2017-01-30 11:19:00  109.455
17535 2017-01-30 12:19:00  109.456
17536 2017-01-30 13:19:00  109.457
17537 2017-01-30 14:19:00  109.459

[17329 rows x 2 columns]
                         G10  River
Date                               
2015-01-30 13:00:00  110.722  261.0
2015-01-30 14:00:00  110.720  260.0
2015-01-30 15:00:00  110.718  260.0
2015-01-30 16:00:00  110.715  260.0
2015-01-30 17:00:00  110.714  260.0
...                      ...    ...
2017-01-30 10:00:00  109.457  135.0
2017-01-30 11:00:00  109.456  136.0
2017-01-30 12:00:00  109.455  138.0
2017-01-30 13:00:00  109.456  141.0
2017-01-30 14:00:00  109.457  144.0

[17546 rows x 2 columns]


'\nhas_nan = resampled_data.isna().any().any()\nprint(has_nan)\ntotal_nan_count = resampled_data.isna().sum().sum()\nprint(total_nan_count)\nnan_per_column = resampled_data.isna().any()\nprint(nan_per_column)\nnan_count_per_column = resampled_data.isna().sum()\nprint(nan_count_per_column)'

In [5]:

n_steps = 24

# Assume merged_data is a DataFrame with 'Date' as index
merged_data_copy = merged_data.copy()
merged_data_copy.reset_index(inplace=True)
dates = merged_data_copy['Date']

# Split the merged data (70% training, 30% testing)
train_size = int(len(merged_data) * 0.7)
merged_data_train = merged_data[:train_size]
merged_data_test = merged_data[train_size:]

# Split dates corresponding to the data split
dates_train = dates[:train_size]
dates_test = dates[train_size:]

# Apply MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(merged_data_train)
test_scaled = scaler.transform(merged_data_test)

# Save the scaler for future use
joblib.dump(scaler, 'scaler.gz')

def create_sequences_with_dates(data, dates, n_steps):
    """
    Create sequences along with corresponding dates.
    """
    X, y, dates_seq = [], [], []
    for i in range(n_steps, len(data)):
        X.append(data[i-n_steps:i, :])
        y.append(data[i, 0])
        # Capture the full range of dates for each sequence
        dates_seq.append(dates.iloc[i - n_steps + 1 : i + 1])
    return np.array(X), np.array(y), np.array(dates_seq)

# Create sequences for training and testing sets
X_train, y_train, dates_train_seq = create_sequences_with_dates(train_scaled, dates_train, n_steps)
X_test, y_test, dates_test_seq = create_sequences_with_dates(test_scaled, dates_test, n_steps)

# Print shapes of X_train, y_train, X_test, y_test for verification
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)
print(dates_test_seq)

X_train shape: (12106, 24, 2)
y_train shape: (12106,)
X_test shape: (5175, 24, 2)
y_test shape: (5175,)
[['2016-06-19T16:13:00.000000000' '2016-06-19T17:13:00.000000000'
  '2016-06-19T18:13:00.000000000' ... '2016-06-20T13:13:00.000000000'
  '2016-06-20T14:13:00.000000000' '2016-06-20T15:13:00.000000000']
 ['2016-06-19T17:13:00.000000000' '2016-06-19T18:13:00.000000000'
  '2016-06-19T19:13:00.000000000' ... '2016-06-20T14:13:00.000000000'
  '2016-06-20T15:13:00.000000000' '2016-06-20T16:13:00.000000000']
 ['2016-06-19T18:13:00.000000000' '2016-06-19T19:13:00.000000000'
  '2016-06-19T20:13:00.000000000' ... '2016-06-20T15:13:00.000000000'
  '2016-06-20T16:13:00.000000000' '2016-06-20T17:13:00.000000000']
 ...
 ['2017-01-29T13:19:00.000000000' '2017-01-29T14:19:00.000000000'
  '2017-01-29T15:19:00.000000000' ... '2017-01-30T10:19:00.000000000'
  '2017-01-30T11:19:00.000000000' '2017-01-30T12:19:00.000000000']
 ['2017-01-29T14:19:00.000000000' '2017-01-29T15:19:00.000000000'
  '2017-01-29

In [ ]:
print(X_train, X_train.shape)

[[[0.8510427  0.66666667]
  [0.85004965 0.66287879]
  [0.8490566  0.66287879]
  ...
  [0.83366435 0.65909091]
  [0.8326713  0.65909091]
  [0.83118173 0.66287879]]

 [[0.85004965 0.66287879]
  [0.8490566  0.66287879]
  [0.84756703 0.66287879]
  ...
  [0.8326713  0.65909091]
  [0.83118173 0.66287879]
  [0.83118173 0.66666667]]

 [[0.8490566  0.66287879]
  [0.84756703 0.66287879]
  [0.84707051 0.66287879]
  ...
  [0.83118173 0.66287879]
  [0.83118173 0.66666667]
  [0.82969215 0.67045455]]

 ...

 [[0.32820258 0.36363636]
  [0.3306852  0.35984848]
  [0.3346574  0.35984848]
  ...
  [0.36494538 0.36363636]
  [0.36693148 0.35984848]
  [0.36842105 0.35984848]]

 [[0.3306852  0.35984848]
  [0.3346574  0.35984848]
  [0.3366435  0.35606061]
  ...
  [0.36693148 0.35984848]
  [0.36842105 0.35984848]
  [0.36891758 0.35606061]]

 [[0.3346574  0.35984848]
  [0.3366435  0.35606061]
  [0.33912612 0.35227273]
  ...
  [0.36842105 0.35984848]
  [0.36891758 0.35606061]
  [0.36991063 0.35606061]]] (12106, 24

In [ ]:
print(X_test, X_test.shape)

[[[0.37239325 0.35227273]
  [0.37288977 0.35227273]
  [0.3733863  0.35227273]
  ...
  [0.38331678 0.34848485]
  [0.38381331 0.34848485]
  [0.38430983 0.34469697]]

 [[0.37288977 0.35227273]
  [0.3733863  0.35227273]
  [0.37487587 0.35227273]
  ...
  [0.38381331 0.34848485]
  [0.38430983 0.34469697]
  [0.38480636 0.34469697]]

 [[0.3733863  0.35227273]
  [0.37487587 0.35227273]
  [0.37586892 0.35227273]
  ...
  [0.38430983 0.34469697]
  [0.38480636 0.34469697]
  [0.38480636 0.34090909]]

 ...

 [[0.2428004  0.21212121]
  [0.2428004  0.20833333]
  [0.24230387 0.20454545]
  ...
  [0.22293942 0.18939394]
  [0.2224429  0.19318182]
  [0.22194638 0.20075758]]

 [[0.2428004  0.20833333]
  [0.24230387 0.20454545]
  [0.24180735 0.1969697 ]
  ...
  [0.2224429  0.19318182]
  [0.22194638 0.20075758]
  [0.2224429  0.21212121]]

 [[0.24230387 0.20454545]
  [0.24180735 0.1969697 ]
  [0.2408143  0.19318182]
  ...
  [0.22194638 0.20075758]
  [0.2224429  0.21212121]
  [0.22293942 0.22348485]]] (5175, 24,

In [ ]:
print(y_train, y_train.shape)

[0.83118173 0.82969215 0.82969215 ... 0.36891758 0.36991063 0.3714002 ] (12106,)


In [ ]:
print(y_test, y_test.shape)

[0.38480636 0.38480636 0.38629593 ... 0.2224429  0.22293942 0.22393247] (5175,)


In [6]:
input_shape=(X_train.shape[1], X_train.shape[2])

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.regularizers import l1_l2

model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=input_shape,   #input shape => (timesteps, features).  X => (samples, timesteps, features)
         dropout=0.2, recurrent_dropout=0.2,
         kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
    Dropout(0.2),
    LSTM(units=50, dropout=0.2, recurrent_dropout=0.2),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=64)


Epoch 1/50
190/190 [==============================] - 21s 79ms/step - loss: 0.1567
Epoch 2/50
190/190 [==============================] - 16s 82ms/step - loss: 0.0104
Epoch 3/50
190/190 [==============================] - 15s 78ms/step - loss: 0.0065
Epoch 4/50
190/190 [==============================] - 15s 80ms/step - loss: 0.0058
Epoch 5/50
190/190 [==============================] - 15s 77ms/step - loss: 0.0049
Epoch 6/50
190/190 [==============================] - 15s 79ms/step - loss: 0.0046
Epoch 7/50
190/190 [==============================] - 14s 75ms/step - loss: 0.0046
Epoch 8/50
190/190 [==============================] - 14s 74ms/step - loss: 0.0045
Epoch 9/50
190/190 [==============================] - 14s 75ms/step - loss: 0.0041
Epoch 10/50
190/190 [==============================] - 14s 75ms/step - loss: 0.0041
Epoch 11/50
190/190 [==============================] - 14s 74ms/step - loss: 0.0038
Epoch 12/50
190/190 [==============================] - 16s 87ms/step - loss: 0.0038
E

In [7]:
model.save('lstmmodel.h5')
model = load_model('lstmmodel.h5')
scaler = joblib.load('scaler.gz')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
# Function to retrain the model with new data
def retrain_model(model, X_new, y_new, epochs=5, batch_size=32):
    # Partial retraining of the model with new data
    model.fit(X_new, y_new, epochs=epochs, batch_size=batch_size, shuffle=True)
    return model

'''# Function to update y_train and recalculate statistics
def update_training_data(new_data, y_train, max_length=10000):
    updated_y_train = np.append(y_train, new_data)
    if len(updated_y_train) > max_length:
        updated_y_train = updated_y_train[-max_length:]
    mean_y_train = np.mean(updated_y_train)
    std_y_train = np.std(updated_y_train)
    return updated_y_train, mean_y_train, std_y_train '''

'# Function to update y_train and recalculate statistics\ndef update_training_data(new_data, y_train, max_length=10000):\n    updated_y_train = np.append(y_train, new_data)\n    if len(updated_y_train) > max_length:\n        updated_y_train = updated_y_train[-max_length:]\n    mean_y_train = np.mean(updated_y_train)\n    std_y_train = np.std(updated_y_train)\n    return updated_y_train, mean_y_train, std_y_train '

In [10]:
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import numpy as np
import pandas as pd

window_size = 24

actual_values = []
predicted_values = []
errors = []
dates_for_plotting = []  # Store dates for plotting

fig, ax = plt.subplots(figsize=(10, 6))

for i in range(X_test.shape[0]):
    X_input = X_test[i, :, :]
    X_input = np.reshape(X_input, (1, X_input.shape[0], X_input.shape[1]))

    # Make a prediction
    forecast = model.predict(X_input)
    dummy = np.zeros((1, 2))
    dummy[:, 0] = forecast[:, 0]
    forecasted_value = scaler.inverse_transform(dummy)[0, 0]
    predicted_values.append(forecasted_value)

    actual = y_test[i] if i < len(y_test) else None
    if actual is not None:
        actual_transformed = scaler.inverse_transform([[actual, 0]])[0, 0]
        actual_values.append(actual_transformed)

    error = abs(forecasted_value - actual_transformed)
    errors.append(error)

    # Convert the last date of each sequence to datetime for plotting
    last_date = pd.to_datetime(dates_test_seq[i][-1])
    dates_for_plotting.append(last_date)

    ax.clear()
    ax.plot(dates_for_plotting, actual_values, label='Observed', color='blue')
    ax.plot(dates_for_plotting, predicted_values, label='Predicted', color='red')
    ax.set_title('G10')
    ax.set_xlabel('Date')
    ax.set_ylabel('Water Level')
    ax.legend()

    if len(errors) >= window_size:
        windowed_errors = errors[-window_size:]
        mean_error = np.mean(windowed_errors)
        std_error = np.std(windowed_errors)
        threshold = mean_error + 3 * std_error

        windowed_anomalies = [j for j, e in enumerate(windowed_errors) if e > threshold]
        # Anomaly detection and model retraining code...

    display(fig)
    plt.pause(0.5)
    clear_output(wait=True)

plt.close(fig)


KeyboardInterrupt: 